***Import all the necessary librarys***

In [29]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from random import randrange, shuffle
from os import path
import librosa
from librosa.display import specshow
import IPython.display as ipd
from math import floor
import kapre
import time

***Check to see if GPU is available***

In [30]:
if (len(tf.config.experimental.list_physical_devices("GPU")) > 0):
    print("GPU is available!")
    gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.75)
    sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

GPU is available!


Open and read the csv with all the audio information
***Set all the paths and basic formats***

In [31]:
csvPath    = r"F:\labeled_audio_data_old.csv"
specPath   = r"C:\Users\cboen\Documents\Programmierungen\AudioDataScience\Labeled_Specs"
audioPath  = r"F:\Labeled_Audio"
modelPath  = r"F:\Models"
logPath    = r"F:\Logs"

version      = r"6.2" #When you make changes, update the version!!
creationTime = time.strftime("%d.%m.%H.%M.%S", time.localtime())

dataFrame = pd.read_csv(csvPath)
dataDictionary = dataFrame.to_dict()
print(dataDictionary.keys())

sampleRate      = 16000
audioFileLength = 3
spec_hopSize    = 256
spec_blockSize  = 1024
audioFileSize   = (sampleRate * audioFileLength, 1)

dict_keys(['Unnamed: 0', 'filename', 'mainSound', 'length', 'sampleRate', 'quality', 'isCut', 'isMixed', 'isChecked', 'threat', 'salience', 'importance'])


***Create the first batch of training data***
The training and test data will be a list of data containing following informations:
filename, mainSound, audioData, targetValue

In [32]:
numAudiosPerBatch = 2000 #Number of audiofiles to train with per Batch

allFileNames = list(dataDictionary['filename'].values())

training_data = []

dataIndex = 0
numBatches   = floor(len(allFileNames) / numAudiosPerBatch)
testDataSize = len(allFileNames) % numAudiosPerBatch
print("TrainBatches:", numBatches, ", Testdata:", testDataSize)

ids = list(range(0, len(allFileNames)))
shuffle(ids)

def createTrainingData():
    global dataIndex
    print("Get new training data: ", int(dataIndex/numAudiosPerBatch), "/", (numBatches-1), sep = "")
    for i in range(numAudiosPerBatch):
        perc = int( 20 * i / numAudiosPerBatch)
        bar = "[" + "=" * (perc) + ">" + "-" * (19-perc) + "]"
        print(bar, end="\r")
        audioFile = ids[i+dataIndex]
        filename = allFileNames[audioFile]
        #Load the spectrogram
        try:
            audioFilePath = path.join(audioPath, (filename+".wav"))
            audioData, sr = librosa.load(audioFilePath, sr=16000, duration = audioFileLength)
            #Load the target values
            mainSound  = dataDictionary['mainSound'][audioFile]
            targetValue = [0, 0, 0]
            targetValue[0] = dataDictionary['threat'][audioFile]     / 10
            targetValue[1] = dataDictionary['salience'][audioFile]   / 10
            targetValue[2] = dataDictionary['importance'][audioFile] / 10
            training_data.append([filename, mainSound, audioData, targetValue])
        except Exception as e:
            pass
    dataIndex += numAudiosPerBatch
    print()
    print("Done!")

test_data = []

def createTestData():
    for i in range(len(allFileNames)-testDataSize, len(allFileNames)):
        audioFile = ids[i]
        filename = allFileNames[audioFile]
        #Load the spectrogram
        try:
            audioFilePath = path.join(audioPath, (filename+".wav"))
            audioData, sr = librosa.load(audioFilePath, sr=16000, duration = audioFileLength)
            #Load the target values
            mainSound  = dataDictionary['mainSound'][audioFile]
            targetValue = [0, 0, 0]
            targetValue[0] = dataDictionary['threat'][audioFile]     / 10
            targetValue[1] = dataDictionary['salience'][audioFile]   / 10
            targetValue[2] = dataDictionary['importance'][audioFile] / 10
        
            test_data.append([filename, mainSound, audioData, targetValue])
        except:
            pass

createTrainingData()
createTestData()

TrainBatches: 4 , Testdata: 331
Get new training data: 0/3


C:\Users\cboen\anaconda3\envs\AudioDataScience39\lib\site-packages\librosa\util\decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


[===================>]
Done!


***Roughly describe the dataset:***

In [33]:
def describeTrainingData():
    print("Number of audiofiles:", len(training_data))
    print("Lets get the percentage of each Class:")
    categorys = set(dataDictionary['mainSound'].values())
    trainingDataCategorys = [data[1] for data in training_data] 
    for key in categorys:
        print("Key:", key.ljust(17, " "), ", perc.:", round(trainingDataCategorys.count(key)/len(training_data), 3))
    del categorys, trainingDataCategorys

describeTrainingData()

Number of audiofiles: 1999
Lets get the percentage of each Class:
Key: bicycle_bell      , perc.: 0.001
Key: clock_alarm       , perc.: 0.006
Key: human             , perc.: 0.109
Key: fire              , perc.: 0.008
Key: screaming         , perc.: 0.003
Key: emergency_vehicle , perc.: 0.001
Key: car_horn          , perc.: 0.036
Key: fireworks         , perc.: 0.006
Key: exterior          , perc.: 0.35
Key: natural           , perc.: 0.037
Key: gunshot           , perc.: 0.044
Key: animal            , perc.: 0.142
Key: interior          , perc.: 0.114
Key: siren             , perc.: 0.135
Key: explosion         , perc.: 0.005
Key: crying_baby       , perc.: 0.007


***Sort the list of training data information into input and output data**

In [34]:
train_x=[]
train_y=[]
def sortTrainingData():
    global train_x, train_y
    train_x = [np.pad(data[2], (0, (audioFileSize[0]-len(data[2])))) for data in training_data]
    #train_y = [[data[3], data[4], data[5]] for data in training_data]
    train_y = [data[-1] for data in training_data]
    train_x = np.asarray(train_x).reshape(-1, audioFileSize[0])
    train_y = np.array(train_y)

sortTrainingData()

Now the really fun part!
***Create a Convolutional network**
...that can get one audiofile as input data and outputs three levels for threat, salience and importance!

In [35]:
model = keras.models.Sequential()
print(audioFileSize)
mel_layer = kapre.composed.get_melspectrogram_layer(input_shape=audioFileSize, n_fft=spec_blockSize, hop_length=spec_hopSize, sample_rate=sampleRate, n_mels=64, mel_f_max=8000.0, return_decibel=True, output_data_format='default', name='melspectrogram')
vgg_layer = tf.keras.applications.VGG16(input_shape=(184,64,1),weights=None, classes=3)# 
vgg_layer.layers[-1].activation=None

model.add(mel_layer)
model.add(vgg_layer)
#model.add(keras.layers.Reshape((3,10)))
#model.add(keras.layers.Activation(keras.activations.linear))

modelName = "stft_vgg16_activation-{}-".format(creationTime) + version
print("Name:", modelName)
print(model.summary())

optimizer = keras.optimizers.SGD(learning_rate = 0.01)
tensorboard = tf.keras.callbacks.TensorBoard(path.join(logPath, modelName), histogram_freq=1)
model.compile(loss="mean_squared_error", optimizer=optimizer, metrics=["accuracy"])

(48000, 1)
Name: stft_vgg16_activation-16.03.14.14.33-6.2
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
melspectrogram (Sequential)  (None, 184, 64, 1)        0         
_________________________________________________________________
vgg16 (Functional)           (None, 3)                 52482755  
Total params: 52,482,755
Trainable params: 52,482,755
Non-trainable params: 0
_________________________________________________________________
None


Now lets get the ***training session*** done!

In [36]:
for batch in range((numBatches-1)):
    try:
        model.fit(train_x, train_y, validation_split=0.05, epochs=10, callbacks=[tensorboard])
    except Exception as e:
        print("Something went wrong!")
        print(e)
    training_data = []
    createTrainingData()
    sortTrainingData()

try:
    model.fit(train_x, train_y, validation_split=0.05, epochs=10, callbacks=[tensorboard])
except Exception as e:
    print("Something went wrong!")
    print(e)

Epoch 1/10
60/60 [==============================] - 8s 118ms/step - loss: 0.1136 - accuracy: 0.8968 - val_loss: 0.0869 - val_accuracy: 0.9700
Epoch 2/10
60/60 [==============================] - 5s 80ms/step - loss: 0.0949 - accuracy: 0.9279 - val_loss: 0.1152 - val_accuracy: 0.9700
Epoch 3/10
60/60 [==============================] - 5s 80ms/step - loss: 0.0908 - accuracy: 0.9279 - val_loss: 0.0824 - val_accuracy: 0.9700
Epoch 4/10
60/60 [==============================] - 5s 80ms/step - loss: 0.0898 - accuracy: 0.9279 - val_loss: 0.0751 - val_accuracy: 0.9700
Epoch 5/10
60/60 [==============================] - 5s 79ms/step - loss: 0.0873 - accuracy: 0.9279 - val_loss: 0.0738 - val_accuracy: 0.9700
Epoch 6/10
60/60 [==============================] - 5s 80ms/step - loss: 0.0837 - accuracy: 0.9279 - val_loss: 0.0852 - val_accuracy: 0.9700
Epoch 7/10
60/60 [==============================] - 5s 79ms/step - loss: 0.0819 - accuracy: 0.9279 - val_loss: 0.0682 - val_accuracy: 0.9700
Epoch 8/10
6

***Lets let the model predict some data!***

In [49]:
i = randrange(dataIndex, len(allFileNames), 1)
audioFile = ids[i]
filename = allFileNames[audioFile]
audioFilePath = path.join(audioPath, (filename+".wav"))
audioData, sr = librosa.load(audioFilePath, sr=16000, duration = audioFileLength)
audioData = np.pad(audioData, (0, (audioFileSize[0]-len(audioData))))
test_x = np.array(audioData).reshape(-1, audioFileSize[0])
mainSound = dataDictionary['mainSound'][ids[i]]
targetValue = [[0 for i in range(10)], [0 for i in range(10)], [0 for i in range(10)], ]
targetValue = [0, 0, 0]
targetValue[0] = dataDictionary['threat'][audioFile]     / 10
targetValue[1] = dataDictionary['salience'][audioFile]   / 10
targetValue[2] = dataDictionary['importance'][audioFile] / 10
test_y = np.array(targetValue)
print("Infos: ", end="")
print( filename, dataDictionary['mainSound'][ids[i]], sep=" | ")
#print(path.join(audioPath, (filename+".wav")))
print("Goal:  ", test_y)
prediction = model(test_x)
print("Result: ", end="")
#print([round(val, 2) for val in prediction.numpy()[0]])
#print("max Val:", np.argmax(prediction.numpy()[0]))
print(prediction.numpy()[0])
ipd.Audio(path.join(audioPath, (filename+".wav")))

Infos: 24347-8-0-2 | siren
Goal:   [0.8 0.9 0.9]
Result: [0.7127811  0.8807801  0.79766375]


***Test the model on unseen testing data***

In [50]:
training_data = test_data[:]
sortTrainingData()
model.evaluate(train_x, train_y)

11/11 [==============================] - 1s 59ms/step - loss: 0.0577 - accuracy: 0.8939


[0.05774260312318802, 0.8939393758773804]

Last but not least:
***Save the model.***

In [51]:
model.save(path.join(modelPath, modelName))

INFO:tensorflow:Assets written to: F:\Models\stft_vgg16_activation-16.03.14.14.33-6.2\assets
